In [1]:
import subprocess
import sys
import os

def install(package) -> None:
    '''
    Helper function to install packages that are not in the runtime environment yet
    '''
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import re
import ftplib
try:
    from astropy.io import fits
except:
    install('astropy')
    from astropy.io import fits
    
from helperFunctions import *


## Change User name and password:
This is the ftp login information and address

In [ ]:
######################################
#      Your values here:
user_name = ""
password = ""
address =  "ftp.caha.es"
#
#######################################

## Establish an FTP connection to the address supplied above:

In [ ]:
ftp = ftplib.FTP(address)
try:
    ftp.login(user_name, password)
except:
    print("ERROR: login authentification failed")


In [ ]:
ftp.pwd()
ftp.retrlines('LIST') 
files = ftp.dir()
print(files)

## Define regular expression matching *-vis_A.fits, *-vis_B.fits, nir_A.fits, nir_B.fits:

Modify the 'target' variable to match different patterns:

In [ ]:
#Regular Expression to match pattern:
target = '.+-(vis|nir)_[AB].fits$'


## Search a remote directory for files containing the 'target' string and retrieve them:

Modify the 'folder' variable to search different directories:

In [ ]:
########################################
#      Insert directory to search here:
folder = './Example_Directory/' 
#
######################################

#list of files matching target regex contained in remote folder:
files = get_file_names(ftp, folder, target)

#retrieve matching files:
retrieve(ftp, files)





## To get variables from all matched files:

Modify 'target_variables' list to desired keyword names. 'target_variables' should be a list, even if only one variable is to be extracted.

This will search all the found files. If only a specific subset of files is to be searched, use the next cell.

In [8]:
###########################################
#    Change This to the desired keyword variables:
target_variables = ['EXAMPLE', 'EXAMPLE2', 'etc']
target_variables = ['HIERARCH CARACAL FOX SNR 7', 'HIERARCH CARACAL FOX SNR 4']


#
###########################################

#initialize the data list:
extracted_data = []

#get file names:
matched_files = os.listdir("./retrieved_fits_files/")

#extract target variable from header of each fits file:
for index, variable in enumerate(target_variables):
    extracted_data.append([])
    [extracted_data[index].append(extract_data_from_fits_header(file, variable)) for file in matched_files]
    
print(extracted_data)



SIMPLE  =                    T /Primary Header created by MWRFITS v1.12         BITPIX  =                    8 /                                                NAXIS   =                    0 /                                                EXTEND  =                    T / FITS dataset may contain extensions            ECHVERS =              1.00000 / version number of echelle data format          COMMENT   FITS (Flexible Image Transport System) format is defined in 'AstronomyCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H COMMENT   FITS (Flexible Image Transport System) 2010A&A...524..A42P            DETSEC  = '[1:4096,1:2048]'    / [px] xrange and yrange of window               DETSIZE = '[1:4096,1:2048]'    / [px] x-range, yrange of full frame             FILENAME= 'car-20220301T18h37m47s-cal-nir.fits' / Filename by 1st stage pipel   OBJECT  = 'DARK    '           / telescope target                               DATE-OBS= '2022-03-01T18:37:47' / [d] UT

## To get variables from only one or several matched files:

Modify 'target_variables' list to desired keyword names. 'target_variables' should be a list, even if only one variable is to be extracted.

Modify the 'target_files' list to change which files to search. Similarly, it should also be a list even if only one file is to be searched.

In [7]:
###########################################
#    Change This to the desired keyword variables:
target_variables = ['HIERARCH CARACAL FOX SNR 7', 'HIERARCH CARACAL FOX SNR 4']
#
###########################################

###########################################
#    Add file names you want to access to the list:
target_files = ['example_file_1.fits', 'example_file_2.fits'] 
#
###########################################

#initialize the data list:
extracted_data = []

#extract target variable from header of each fits file:
for index, variable in enumerate(target_variables):
    extracted_data.append([])
    [extracted_data[index].append(extract_data_from_fits_header(file, variable)) for file in target_files]
    
print(extracted_data)




[['Error: could not access ./retrieved_fits_files/example_file_1.fits', 'Error: could not access ./retrieved_fits_files/example_file_2.fits'], ['Error: could not access ./retrieved_fits_files/example_file_1.fits', 'Error: could not access ./retrieved_fits_files/example_file_2.fits']]


## Save the data in CSV format:

This will take the variables extracted by the previous cells and save them in a csv file with KEYWORDs as column names in the header row. Change the 'output_file' variable if you want to save the csv to a different filename.

In [4]:
##########################################
# Modify this variable if another output filename is needed:
output_file = 'output.csv'
#
##########################################



with open(output_file, 'w') as file:
    #get number of variables:
    num_vars = len(target_variables)-1
    #write header column with var names
    [file.write(f'{variable},') for index, variable in enumerate(target_variables) if index < num_vars]
    #write last column with newline:
    file.write(f'{target_variables[num_vars]}\n')
    #write each line, ending with newline instead of comma:
    for line_number in range(len(extracted_data[0])):
        [file.write(f'{variable[line_number]},') for index, variable in enumerate(extracted_data) if index < num_vars]
        file.write(f'{extracted_data[num_vars][line_number]}\n')



# Plotting the data

## Import plotting packages

In [ ]:
try:
    import matplotlib.pyplot as pltf
except:
    install('matplotlib')
    import matplotlib

try:
    import seaborn as sns
except:
    install('seaborn')
    import seaborn
    
%matplotlib inline

## Read in the csv: